In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.ne

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-30 02:43:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-12-30 02:43:16 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [7]:
# Get the number of rows in the DataFrame.
user_data_df.count()


5101693

# Transform the Data

## Create the "review_id_table".

In [8]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = user_data_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show()



+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
|R30DKW1GJWLPZC|   50484904|B004EWGS5G|     442453110| 2013-09-09|
|R18DPFG2FALJI9|    7145636|B00BNRJAT6|     856774152| 2013-09-09|
|R24D677N5WBW5Q|    6285538|B007FZOXJM|       5589837| 2013-09-09|
|R2FCJ9BQLSIOR3|   10278048|B00B6AK7LU|     362701357| 2013-09-09|
|R1R6K4MAKDWTXI|   16568972|B00EVMMLU0|     342745087| 2013-09-09|
|R3R5DILCWM8J7B|   26001763|B00E4W4984|     186117550| 2013-09-09|
| RR5K72IZOCOFE|   48203259|B005A1JBB8|     947574172| 2013-09

## Create the "products" Table

In [9]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = user_data_df.select(['product_id', 'product_title']).dropDuplicates(['product_id'])
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|068986843X|The Stolen Relic ...|
|B000EOTFRQ|  The Grand Alliance|
|B000F83SV0| Lavender Excursions|
|B000F83SWO| Season of the Witch|
|B000F83T2S|   No Enemy But Time|
|B000F83T3C|The Return of Tar...|
|B000F83T82|              Terror|
|B000F83TDW|    Unicorn Mountain|
|B000F83TEQ|Gold Coffin Murde...|
|B000FA5KFA|          The Prince|
|B000FA5KKA|Interplanetary Hu...|
|B000FA5KSC|Winning the Loser...|
|B000FA5KTQ|Buying Travel Ser...|
|B000FA5KX2|    The McKinsey Way|
|B000FA5L18|Motivating Employ...|
|B000FA5LMM|The Post-Traumati...|
|B000FA5LPE|The Disney Way, R...|
|B000FA5LQ8|Leading Self-Dire...|
|B000FA5LT0|The Unoffcial Bus...|
|B000FA5LTA|Schaum's Easy Out...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [10]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = user_data_df.groupby('customer_id').agg({'customer_id': 'count'})
customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   27134317|               306|
|   36063094|                15|
|   49422311|                 1|
|   18978114|                 1|
|   13455482|                 2|
|   32153206|                 1|
|   10274872|                 4|
|   20977654|                 1|
|   48042607|                 1|
|   11376183|                 7|
|   31762963|                 1|
|   11481508|                 4|
|   12362676|                 2|
|   31176171|                10|
|   44699860|                23|
|   52734389|                 8|
|   24303040|                 3|
|   26322451|                77|
|   14521576|                30|
|   44269342|                10|
+-----------+------------------+
only showing top 20 rows



## Create the "vine_table".

In [11]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns.  

vine_df = user_data_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RGYFDX8QXKEIR|          4|            0|          0|   N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|
| R7DRFHC0F71O0|          5|            0|          0|   N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|
|R1VXTPUYMNU687|          5|            1|          2|   N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|
|R18DPFG2FALJI9|          5|            0|          0|   N|
|R24D677N5WBW5Q|          5|            0|          0|   N|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|
|R3R5DILCWM8J7B|          5|            0|          0|   N|
| RR5K72IZOCOFE|          4|            0|          0|   N|
|R3K9PJU5GLDY3O|          5|            1|          2|   N|
|R1KTZMCDOJXAEK|          5|            

# Load

In [14]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cp5sm1crlurk.us-west-1.rds.amazonaws.com:5432/my_aws_db"
config = {"user":"postgres", "password": "postgres", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='reviewid_table', mode=mode, properties=config)


In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)


In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)


In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)